In [1]:
!pip install pysiology

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#@title Fix
import os,shutil
os.remove("/usr/local/lib/python3.9/dist-packages/pysiology/electromyography.py")
shutil.move("/content/electromyography.py","/usr/local/lib/python3.9/dist-packages/pysiology/electromyography.py")

'/usr/local/lib/python3.9/dist-packages/pysiology/electromyography.py'

In [3]:
! wget -r -N -c -np https://physionet.org/files/gaitndd/1.0.0/

--2023-04-19 15:48:57--  https://physionet.org/files/gaitndd/1.0.0/
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘physionet.org/files/gaitndd/1.0.0/index.html’

physionet.org/files     [ <=>                ]  36.84K  --.-KB/s    in 0.08s   

Last-modified header missing -- time-stamps turned off.
2023-04-19 15:48:58 (445 KB/s) - ‘physionet.org/files/gaitndd/1.0.0/index.html’ saved [37721]

Loading robots.txt; please ignore errors.
--2023-04-19 15:48:58--  https://physionet.org/robots.txt
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 22 [text/plain]
Saving to: ‘physionet.org/robots.txt’

physionet.org/robot 100%[===================>]      22  --.-KB/s    in 0s      

2023-04-19 15:48:59 (786 KB/s) - ‘physionet.org/robots.txt’ saved [22/22]

--2023-04-19 15:

In [4]:
import pysiology
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Thank you for using Pysiology. If you use it in your work, please cite:
Gabrieli G., Azhari A., Esposito G. (2020) PySiology: A Python Package for Physiological Feature Extraction. In: Esposito A., Faundez-Zanuy M., Morabito F., Pasero E. (eds) Neural Approaches to Dynamics of Signal Exchanges. Smart Innovation, Systems and Technologies, vol 151. Springer, Singapore. https://doi.org/10.1007/978-981-13-8950-4_35


In [5]:
#EDA
from collections import Counter
# data preprocessing
from sklearn.preprocessing import StandardScaler
# data splitting
from sklearn.model_selection import train_test_split
# data modeling
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import Perceptron
# from mlxtend.classifier import StackingCVClassifier

In [6]:
data_1 = pd.read_csv("physionet.org/files/gaitndd/1.0.0/als1.ts",delimiter="\t",names=["Elapsed Time (sec)","Left Stride Interval (sec)","Right Stride Interval (sec)","Left Swing Interval (sec)",
                                                      "Right Swing Interval (sec)","Left Swing Interval (% of stride)","Right Swing Interval (% of stride)",
                                                      "Left Stance Interval (sec)","Right Stance Interval (sec)","Left Stance Interval (% of stride)",
                                                      "Right Stance Interval (% of stride)","Double Support Interval (sec)","Double Support Interval (% of stride)"])

data_1.head()

,Elapsed Time (sec),Left Stride Interval (sec),Right Stride Interval (sec),Left Swing Interval (sec),Right Swing Interval (sec),Left Swing Interval (% of stride),Right Swing Interval (% of stride),Left Stance Interval (sec),Right Stance Interval (sec),Left Stance Interval (% of stride),Right Stance Interval (% of stride),Double Support Interval (sec),Double Support Interval (% of stride)
0,22.3200,1.2833,1.3533,0.4067,0.4133,31.69,30.54,0.8767,0.9400,68.31,69.46,0.4633,36.10
1,23.6433,1.3233,1.2667,0.4833,0.4000,36.52,31.58,0.8400,0.8667,63.48,68.42,0.4400,33.25
2,24.9467,1.3033,1.3600,0.4500,0.4267,34.53,31.37,0.8533,0.9333,65.47,68.63,0.4267,32.74
3,26.3633,1.4167,1.2833,0.5033,0.3667,35.53,28.57,0.9133,0.9167,64.47,71.43,0.5467,38.59
4,27.6000,1.2367,1.4067,0.3467,0.3567,28.03,25.36,0.8900,1.0500,71.97,74.64,0.5333,43.13


In [7]:
dataset_path = "physionet.org/files/gaitndd/1.0.0"
dict_data = {'MNF': [], 'MDF': [],'PeakFrequency': [], 'MNP': [],'TTP': [], 'SM1': [],'SM2': [], 'SM3': []
  ,'FR': [], 'PSR': [], 'VCF': [],'IEMG': [], 'MAV': [], 'MAV1': [], 
 'MAV2': [], 'SSI': [], 'VAR': [], 'TM3': [], 'TM4': [], 'TM5': [],  
 'LOG': [], 'RMS': [], 'WL': [], 'AAC': [], 'DASDV': [], 'ZC': [], 'MYOP':[],   
 'WAMP': [], 'SSC': [], 'MAVSLPk1':[],"MAVSLPk2":[],"MAVSLPk3":[],"label":[]}
  
folders = [os.path.join(dataset_path, folder_name) for folder_name in os.listdir(dataset_path) if str(folder_name).endswith(".ts")]  
  
labels = ["als","control","hunt","park"]
for folder in folders:  
  if(re.split("\d", str(folder.split("/")[4]))[0] != "example"):
    df = pd.read_csv(folder,delimiter="\t",names=["Elapsed Time (sec)","Left Stride Interval (sec)","Right Stride Interval (sec)","Left Swing Interval (sec)",  
                                                      "Right Swing Interval (sec)","Left Swing Interval (% of stride)","Right Swing Interval (% of stride)",  
                                                      "Left Stance Interval (sec)","Right Stance Interval (sec)","Left Stance Interval (% of stride)",  
                                                      "Right Stance Interval (% of stride)","Double Support Interval (sec)","Double Support Interval (% of stride)"])
    target = ["Left Stride Interval (sec)","Right Stride Interval (sec)","Left Swing Interval (sec)","Right Swing Interval (sec)",
              "Left Stance Interval (sec)","Right Stance Interval (sec)","Double Support Interval (sec)"] 
    label = labels.index(re.split("\d", str(folder.split("/")[4]))[0]) 
    for f in target:
      data = df[f]
      analyzedEMG = pysiology.electromyography.analyzeEMGnew(data,samplerate=200,preprocessing=False) 
      time,freq = analyzedEMG

      freq = analyzedEMG[freq]
      keys1 = list(freq.keys())

      for i in keys1 :
        dict_data[i].append(freq[i])

      time = analyzedEMG[time]
      keys2 = list(time.keys())[:-1]

      for i in keys2 :
        dict_data[i].append(time[i])

      dict_data["MAVSLPk1"].append(time["MAVSLPk"][0])
      dict_data["MAVSLPk2"].append(time["MAVSLPk"][1])
      dict_data["MAVSLPk3"].append(time["MAVSLPk"][2])
      dict_data["label"].append(label)

dataFrame = pd.DataFrame(dict_data)

/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 183, using nperseg = 183
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/pysiology/electromyography.py:690: RuntimeWarning: divide by zero encountered in double_scalars
  FR = LF / HF
/usr/local/lib/python3.9/dist-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 167, using nperseg = 167
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.9/dist-packages/pysiology/electromyography.py:690: RuntimeWarning: divide by zero encountered in double_scalars
  FR = LF / HF
/usr/local/lib/python3.9/dist-packages/pysiology/electromyography.py:690: RuntimeWarning: divide by zero encountered in double_scalars
  FR = LF / HF
/usr/local/lib/python3.9/dist-packages/pysiology/electromyography.py:690: RuntimeWarning: divide by zero e

In [ ]:
dataFrame

In [ ]:
dataFrame.iloc[:60,:16]

In [ ]:
dataFrame.iloc[:60,16:]

In [ ]:
dataFrame.columns

Index(['MNF', 'MDF', 'PeakFrequency', 'MNP', 'TTP', 'SM1', 'SM2', 'SM3', 'FR',
       'PSR', 'VCF', 'IEMG', 'MAV', 'MAV1', 'MAV2', 'SSI', 'VAR', 'TM3', 'TM4',
       'TM5', 'LOG', 'RMS', 'WL', 'AAC', 'DASDV', 'ZC', 'MYOP', 'WAMP', 'SSC',
       'MAVSLPk1', 'MAVSLPk2', 'MAVSLPk3', 'label'],
      dtype='object')

In [8]:
dataFrame =  dataFrame.drop(columns='FR')
dataFrame =  dataFrame.drop(columns='ZC')
dataFrame =  dataFrame.drop(columns='MYOP')

In [ ]:
# !pip -q install dataprep

In [ ]:

# from dataprep.eda import plot, plot_correlation, plot_missing,create_db_report,create_report
# report = create_report(dataFrame)
# report

In [ ]:
dataFrame["label"].unique()

array([1, 2, 3, 0])

In [ ]:
  als = dataFrame[dataFrame.label == 0]
  control = dataFrame[dataFrame.label==1]
  hunt = dataFrame[dataFrame.label==2]
  park = dataFrame[dataFrame.label==3]

In [ ]:
def plot_scatter(x,y):
  fig, ax = plt.subplots()
  fig.set_size_inches(13, 7) 
  # lables and scatter points
  ax.scatter(als[x], als[y], label="als", facecolor="blue")
  ax.scatter(control[x], control[y], label="control", facecolor="green")
  ax.scatter(hunt[x], hunt[y], label="hunt", facecolor="red")
  ax.scatter(park[x], park[y], label="park", facecolor="Yellow")
  ax.set_xlabel(x)
  ax.set_ylabel(y)
  ax.grid()
  ax.set_title("")
  ax.legend()

In [ ]:
plot_scatter('MNF', 'MDF')

In [ ]:
plot_scatter('WL', 'AAC')

In [ ]:
# plt.figure(figsize=(50,50))
# sns.heatmap(dataFrame.corr())

In [ ]:
data.describe()

count    277.000000
mean       0.266257
std        0.031327
min        0.233300
25%        0.253300
50%        0.260000
75%        0.270000
max        0.633300
Name: Double Support Interval (sec), dtype: float64

# Main function

In [21]:
def Main(dataFrame):
    models = {
        'Logistic Regression':LogisticRegression(max_iter=5000),
        "Naive Bayes":GaussianNB( var_smoothing=1e-018),
        'Support Vector ':SVC(),  
        "Decision Tree":DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(),
        "K-Neighbors":KNeighborsClassifier(n_neighbors=2),
        "Extreme Gradient Boost":XGBClassifier(learning_rate=0.01, n_estimators=25, max_depth=15,gamma=0.6, subsample=0.52,colsample_bytree=0.6,seed=27, reg_lambda=2, booster='dart', colsample_bylevel=0.6, colsample_bynode=0.5),
        "Ada Boost":AdaBoostClassifier(),
        "LDA":LinearDiscriminantAnalysis(),
        "Perceptron":Perceptron()
    }
    columns = list(dataFrame.columns)
    data={  
        "feature":[],
        'Logistic Regression':[],  
        "Naive Bayes":[],
        'Support Vector ':[],   
        "Decision Tree":[],
        "Random Forest": [],  
        "K-Neighbors":[],
        "Extreme Gradient Boost":[],    
        "Ada Boost":[],
        "LDA":[], 
        "Perceptron":[],   
    }  
    models_names = list(models.keys())
    for j in columns[:-1]:  
      copy_model = models.copy()  
      y, X = dataFrame['label'], dataFrame.loc[:,[j]].astype('int64')  
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   
      for i in models_names:      
        copy_model[i].fit(X_train, y_train)      
        predict = copy_model[i].predict(X_test)   
        acc = accuracy_score(y_test, predict)     
        data[i].append(acc*100)  
      data["feature"].append(j)    
    frame =  pd.DataFrame(data)  
    return frame  

In [22]:
Main(dataFrame)

,feature,Naive Bayes,Support Vector,Decision Tree,Random Forest,K-Neighbors,Extreme Gradient Boost,Ada Boost,LDA,Perceptron
0,MNF,41.111111,43.333333,37.777778,36.666667,25.555556,42.222222,44.444444,43.333333,15.555556
1,MDF,45.555556,48.888889,34.444444,35.555556,36.666667,41.111111,42.222222,45.555556,22.222222
2,PeakFrequency,38.888889,37.777778,44.444444,48.888889,44.444444,46.666667,44.444444,38.888889,32.222222
3,MNP,23.333333,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,15.555556
4,TTP,23.333333,30.000000,32.222222,32.222222,32.222222,30.000000,30.000000,30.000000,30.000000
5,SM1,21.111111,32.222222,32.222222,32.222222,35.555556,31.111111,27.777778,30.000000,34.444444
6,SM2,23.333333,30.000000,32.222222,32.222222,32.222222,30.000000,30.000000,30.000000,15.555556
7,SM3,23.333333,30.000000,32.222222,32.222222,32.222222,30.000000,30.000000,30.000000,15.555556
8,PSR,21.111111,37.777778,37.777778,37.777778,37.777778,37.777778,37.777778,30.000000,12.222222
9,VCF,41.111111,43.333333,26.666667,28.888889,24.444444,38.888889,41.111111,42.222222,31.111111
